In [ ]:
!pip install numpy==1.16.2

In [ ]:
import sys
sys.path.append('/kaggle/input/deeplearn')
sys.path.append('/kaggle/input/bertdp')
sys.path.append('/kaggle/input')
sys.path.append('/kaggle/input/tensorr/tensorflow/_api/v1')
import deeppavlov
import tensorflow 

In [ ]:
!cp /kaggle/input/gggggggg/__init__.py /kaggle/input/tensorr/tensorflow/_api/v1/__init__.py

In [ ]:
tensorflow.__version__

In [ ]:
from deeppavlov import build_model, configs

In [ ]:
model = build_model('/kaggle/input/config/config.json')

In [ ]:
import json
import numpy as np 
import pandas as pd
import re
import random
import os
for dirname, _, filenames in os.walk('/kaggle/input/tensorflow2-question-answering/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm_notebook as tqdm
from Levenshtein import ratio as levenshtein_distance

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction import text

from scipy import spatial
from deeppavlov import build_model, configs

In [ ]:
tags = ['is', 'are', 'do', 'does', 'did', 'was', 'were', 'will', 'can', 'the', 'a', 'of', 'in', 'and', 'on', \
         'what', 'where', 'when', 'which' + '<P>', '</P>', '<Table>', '</Table>', '<Tr>', '</Tr>', '<Ul>', '<Ol>', '<Dl>', '</Ul>', '</Ol>', \
             '</Dl>', '<Li>', '<Dd>', '<Dt>', '</Li>', '</Dd>', '</Dt>']

question_words = ['is', 'are', 'do', 'does', 'did', 'was', 'were', 'will', 'can']

In [ ]:
def clean(x):
    x = x.lower()
    for r in tags:
        x = x.replace(r, '')
    x = re.sub(' +', ' ', x)
    return x

def find(start, text, string):
    fnd = string.split(" ")
    res = text.split(" ")
    for i in fnd:
        for j in range(len(res)):
            if i == res[j] and (j + len(fnd) - 1) < len(res) and fnd[-1] == res[j + len(fnd) - 1]:
                return start + j, start + j + len(fnd)
    return None

In [ ]:
def predict(json_data):
    candidates = json_data['long_answer_candidates']
    candidates = [c for c in candidates if c['top_level'] == True]
    doc_tokenized = json_data['document_text'].split(' ')
    question = json_data['question_text'].split(' ') 

    tfidf = TfidfVectorizer(ngram_range=(1,3), stop_words=text.ENGLISH_STOP_WORDS.union(["book"]))
    tfidf.fit([json_data['document_text']])
    q_tfidf = tfidf.transform([json_data['question_text']]).todense()

    scores = []
    count = 1
    for i, c in enumerate(candidates):
        s, e = c['start_token'], c['end_token']
        t = ' '.join(doc_tokenized[s:e])
        t_tfidf = tfidf.transform([t]).todense()
        score = 1 - spatial.distance.cosine(q_tfidf, t_tfidf)
        
        if doc_tokenized[s] == '<P>':
            score += 0.25**count
            count += 1
        scores.append(score)
        
    ans = (np.array(candidates)[np.argsort(scores)])[-1:].tolist()
    
    if np.max(scores) < 0.15:
        long = ['-1:-1']
        ans = [{'start_token': 0, 'end_token': 0}]
    else:
        long = [str(a['start_token']) + ':' + str(a['end_token']) for a in ans]
        answer_short = model([" ".join(doc_tokenized[ans[0]["start_token"]:ans[0]["end_token"]])], [json_data['question_text']])[0][0]
        tmp = find(ans[0]["start_token"]," ".join(doc_tokenized[ans[0]["start_token"]:ans[0]["end_token"]]) ,answer_short)
        if tmp:
            short = str(tmp[0]) + ':' + str(tmp[1])
        else:
            if question[0] in question_words:
                short = 'YES'
            else:
                short = ''
            
    return long, short

In [ ]:
ids = []
preds = []

with open('/kaggle/input/tensorflow2-question-answering/simplified-nq-test.jsonl', 'r') as json_file:
    for line in tqdm(json_file): 
        json_data = json.loads(line)
        l_ans, s_ans = predict(json_data)
        ids += [str(json_data['example_id']) + '_long']*len(l_ans)
        ids.append(str(json_data['example_id']) + '_short')
        preds += l_ans
        preds.append(s_ans) 
        
df = pd.DataFrame()
df['example_id'] = ids
df['PredictionString'] = preds
sub = df[['example_id', 'PredictionString']].groupby('example_id').agg(lambda x: ' '.join(x) if len(x) > 1 else x).reset_index()
sub.to_csv('submission.csv', index=False)

sub.head(10)

In [ ]:
sub